# Logistic Regression

Let's see an example of how to run a logistic regression with Python and Spark! This is documentation example, we will quickly run through this and then show a more realistic example, afterwards, you will have another consulting project!

In [1]:
import findspark
findspark.init('/home/gkouskosv/spark-2.4.5-bin-hadoop2.6/')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logregdoc').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
# Load training data
training = spark.read.format('libsvm').load('sample_libsvm_data.txt')

lr = LogisticRegression()

# Fit the model
lrModel = lr.fit(training)

trainingSummary = lrModel.summary

In [6]:
trainingSummary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [5]:
trainingSummary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [28]:
lr_train, lr_test = training.randomSplit([0.7,0.3])

final_model = LogisticRegression()
fit_final = final_model.fit(lr_train)
prediction_and_labels = fit_final.evaluate(lr_test)

In [11]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[22.7019804271679...|[0.99999999986175...|       0.0|
|  0.0|(692,[100,101,102...|[6.96666635994119...|[0.99905809741943...|       0.0|
|  0.0|(692,[122,123,124...|[21.3610531659296...|[0.99999999947153...|       0.0|
|  0.0|(692,[122,123,148...|[19.9290190804439...|[0.99999999778722...|       0.0|
|  0.0|(692,[123,124,125...|[30.9470131836241...|[0.99999999999996...|       0.0|
|  0.0|(692,[124,125,126...|[31.6248684354497...|[0.99999999999998...|       0.0|
|  0.0|(692,[124,125,126...|[30.7401277929721...|[0.99999999999995...|       0.0|
|  0.0|(692,[124,125,126...|[23.8169358343855...|[0.99999999995466...|       0.0|
|  0.0|(692,[126,127,128...|[31.7266743416726...|[0.99999999999998...|       0.0|
|  0.0|(692,[126

In [12]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, 
                                   MulticlassClassificationEvaluator)

In [34]:
prediction_and_labels.predictions.select(['label','prediction']).show()

AttributeError: 'DataFrame' object has no attribute 'predictions'

## Evaluators

Evaluators will be a very important part of our pipline when working with Machine Learning, let's see some basics for Logistic Regression, useful links:

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator

In [31]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')

In [26]:
# For multiclass
multi_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
                                             metricName='accuracy')

In [32]:
acc = evaluator.evaluate(prediction_and_labels)